In [27]:
import csv
from os import listdir
from os.path import join
import numpy as np
import pandas as pd

tsv_folder_path = './tsvs_for_ensemble'
filelist =[f for f in listdir(tsv_folder_path)]

# thresh_list = [0.515, 0.513, 0.503, 0.506, 0.514]

mode= "vote"

if mode == "mean":
#     thresh = sum(thresh_list)/len(thresh_list)
    thresh = 0.513
    probs_list = []
    for tsv in filelist:
        probs = []
        if "tsv" in tsv and "prob" in tsv:
            with open(join(tsv_folder_path, tsv), 'r') as tsv:
                tsv_file = csv.reader(tsv, delimiter = "\t")
                for row in tsv_file:
                    prob = float(row[0])
                    probs.append(prob)
            probs_list.append(probs)


    num_test = len(probs_list[0]) #9192

    ensemble_probs = [0]*len(probs_list[0])
    for probs in probs_list:
        for i in range(num_test):
            ensemble_probs[i] += probs[i]
            
    



    ensemble_preds[:] = [((x / len(probs_list)) >= thresh)*1 for x in ensemble_probs]
    df = pd.DataFrame(ensemble_preds)
    
elif mode == "vote":
    preds_list = []
    for tsv in filelist:
        if 'tsv' in tsv and "pred" in tsv:
            preds = []
            with open(join(tsv_folder_path, tsv), 'r') as tsv:
                tsv_file = csv.reader(tsv, delimiter = "\t")
                for row in tsv_file:
                    pred = int(row[0])
                    preds.append(pred)
            preds_list.append(preds)
    
    ensemble_preds = []
    for i in range(len(preds_list[0])):
        count = 0
        for j in range(len(preds_list)):
            if preds_list[j][i] == 1:
                count +=1
        if count > len(preds_list)/2:
            ensemble_preds.append(1)
        else:
            ensemble_preds.append(0)
            
            
            
# elif mode == 'Unanimous':
#     preds_list = []
#     for tsv in filelist:
#         if 'tsv' in tsv:
#             idx = int(tsv.split('.')[0][-1])-1
#             thresh = thresh_list[idx]
#             preds = []
#             with open(join(tsv_folder_path, tsv), 'r') as tsv:
#                 tsv_file = csv.reader(tsv, delimiter = "\t")
#                 for row in tsv_file:
#                     prob = float(row[0])
#                     pred = (prob>= thresh) * 1
#                     preds.append(pred)
#             preds_list.append(preds)
            
#     ensemble_preds = []

#     for i in range(len(preds_list[0])):
#         count = 0
#         prediction = preds_list[0][i]
#         for j in range(1, len(preds_list)):
#             if preds_list[j][i] ==prediction:
#                 count+=1
#         if count != len(preds_list)-1:
#             ensemble_preds.append(1)
#         else:
#             ensemble_preds.append(prediction)
            
    
        
    
    df = pd.DataFrame(ensemble_preds)
df.to_csv('ensembled_prediction.tsv', index=False, header=None, sep="\t")


